In [1]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
data = pd.read_csv('data.csv')

#pre-processing to calculate outside good shares
shares = data[['Market_ID','Inside Good Share','PPO']].copy()
shares['PPO Share'] = data['Inside Good Share'] * data['PPO']
shares['HMO Share'] = data['Inside Good Share'] * (1 - data['PPO'])

group_shares = shares.groupby('Market_ID').sum()
group_shares['Outside Good Share'] = 1 - group_shares['Inside Good Share']

data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')

print data.min()

Market_ID             1.000000
Plan_ID               1.000000
PPO                   0.000000
Network Score         0.780000
Satisfaction Score    0.725000
Premium               2.269728
Inside Good Share     0.037670
Outside Good Share    0.096070
dtype: float64


In [3]:
#first estimate using logit
class logit(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

        
    def momcond(self, params):
        #unwrap stuff
        shares = np.array(self.endog).transpose()
        exog = np.array(self.exog)
        instr = np.array(self.instrument)
        
        lshare = np.log(shares[0]) -  np.log(shares[1])
        lshare = lshare.transpose()
       
        lshare_fit = np.matmul(exog,params) #linear equation    
        
        xi = lshare_fit - lshare
        g = instr * xi[:, np.newaxis]
        
        return g

In [4]:
#calculate hausmann insturments
mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
hausman_instr = plan_dum

#set up x and y
y = data[['Inside Good Share','Outside Good Share']]
x =  data[['Network Score','Satisfaction Score','PPO','Premium']]

In [5]:
#set up initial est
beta_init = np.full(len(x.columns),1)

#set up model
model = logit(y , x, hausman_instr)

result = model.fit(beta_init, maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 292
         Function evaluations: 497
Optimization terminated successfully.
         Current function value: 0.005149
         Iterations: 139
         Function evaluations: 249
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        16.99
Model:                                                   logit   Prob (Hansen J):                 0.108
Method:                                                    GMM                                         
Date:                                         Thu, 11 Oct 2018                                         
Time:                                                 08:11:57                                         
No. Observations:                                         3300                                  

In [6]:
#correct way to compute shares...

def comp_shares(x,p,alpha,beta,labels=False):
    #compute exp(delta_j)
    x = x.copy()
    x['exp_delta'] =  np.exp ( np.matmul(x[['Network Score','Satisfaction Score','PPO']],beta) - alpha*p )
    #print x['exp_delta']
    
    #compute 1 + sum_j exp(delta_j)
    sum_delta = x.groupby('Market_ID').sum()
    sum_delta['sum_exp_delta'] = 1 + sum_delta['exp_delta'] 
    
    x = pd.merge(x, sum_delta[['sum_exp_delta']], 
                right_index=True, left_on = 'Market_ID')

    #compute s_j
    x['fitted_share'] = x['exp_delta']/x['sum_exp_delta']
    
    if labels: 
        return x[['Plan_ID','Market_ID','fitted_share']]
    
    return np.array(x['fitted_share']).squeeze()
        

#set up parameters 
observ = data[['Plan_ID','Market_ID','Network Score','Satisfaction Score','PPO']]
prices = np.array( data['Premium'] ) #needed for later...

beta = result.params[:-1]
alpha = abs(result.params[3])

shares = comp_shares(observ,prices,alpha,beta)

print shares

[0.17502021 0.23839869 0.29850248 ... 0.09512126 0.09887772 0.13083635]


In [7]:
def comp_elasticity(shares, prices, alpha):
    #set up matrix skeleton
    own_price = np.identity(len(shares))
    cross_price = 1 - own_price

    #actually calculate elasticity
    cross_elasticity = shares * alpha * prices
    own_elasticity  = -(1-shares) * alpha * prices

    return cross_price*cross_elasticity + own_price *own_elasticity


#agregate elasticities
elasticity = comp_elasticity(shares, prices, alpha)
print elasticity[1][1],elasticity[2][1],elasticity[3][1] #second index tells you wrt what good i.e. denom


#average elasticity
avg_shares =  np.array( comp_shares(observ,prices,alpha,beta,
                                    labels=True).groupby('Plan_ID').mean()['fitted_share']).squeeze()
avg_price = np.array( data[['Plan_ID','Premium']].groupby('Plan_ID').mean() ).squeeze()
avg_elasticity = comp_elasticity(avg_shares, avg_price, alpha)
print avg_elasticity

-3.8405528281901504 1.2021812009896957 1.2021812009896957
[[-4.32655865  0.5554553   1.08916071  0.82684009  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942 -4.3085966   1.08916071  0.82684009  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942  0.5554553  -4.2870623   0.82684009  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942  0.5554553   1.08916071 -4.37731375  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942  0.5554553   1.08916071  0.82684009 -4.25288515  0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.0026494

In [8]:
#solve for marginal costs
def comp_markup(shares):
    shares_vector = np.array([shares])
    
    #set up matrix    
    own_price = np.identity(len(shares))

    #caclulate formula
    own_deriv  = - alpha * (1-shares)  * shares
    
    derivative = own_price *own_deriv
    #take inverse and calc markup
    inv_derivative = np.linalg.inv(derivative)

    markup = - np.matmul(inv_derivative, shares_vector.transpose()) 
    return markup.transpose()[0]


markup = comp_markup(shares)
mc = prices - markup
print markup
print mc

[0.58800458 0.63693679 0.69150906 ... 0.53608497 0.53831972 0.55811342]
[1.90975802 1.80925261 2.10367754 ... 2.12844873 1.81858468 1.96851598]


In [9]:
def industry_profits(p, rebate=0, x=observ, beta=beta):
    """computes agregate profits"""
    shares = comp_shares(x, p, alpha, beta)
    return np.matmul(shares, np.array([prices - mc + rebate]).transpose())

In [16]:
#iterate through to solve profits
new_prices = np.empty(len(prices))

for i in range(len(prices)): 
    
    def profits(p, all_prices=prices.copy(), x=observ, mc = mc, beta=beta, rebate=.25,):
        all_prices[i] = p
        shares = comp_shares(x, all_prices, alpha, beta)
        return - shares[i]*(p - mc[i] + rebate)      
    
    res = minimize(profits, prices[i], method='BFGS', options={'disp': False})
    new_prices[i]=res.x

new_prices = np.array(new_prices).squeeze()
np.savetxt("prices.csv", new_prices, delimiter=",")

In [ ]:
#profits per enrollee
#outside good shares
def comp__outside_shares(x,p,alpha,beta,labels=False):
    #compute exp(delta_j)
    x = x.copy()
    x['exp_delta'] =  np.exp ( np.matmul(x[['Network Score','Satisfaction Score','PPO']],beta) - alpha*p )
    #print x['exp_delta']
    
    #compute 1 + sum_j exp(delta_j)
    sum_delta = x.groupby('Market_ID').sum()
    sum_delta['sum_exp_delta'] = 1 + sum_delta['exp_delta'] 
    
    x = pd.merge(x, sum_delta[['sum_exp_delta']], 
                right_index=True, left_on = 'Market_ID')

    #compute s_0
    x['outside_fitted_share'] = 1/x['sum_exp_delta']
    
    return x[['Plan_ID','Market_ID','outside_fitted_share']]
    

        

#set up parameters 
observ = data[['Plan_ID','Market_ID','Network Score','Satisfaction Score','PPO']]
prices = np.array( data['Premium'] ) #needed for later...

beta = result.params[:-1]
alpha = abs(result.params[3])

shares = comp__outside_shares(observ,prices,alpha,beta)

print(shares[['outside_fitted_share']].mean())
        

#set up parameters 
observ = data[['Plan_ID','Market_ID','Network Score','Satisfaction Score','PPO']]
prices = new_prices #needed for later...

beta = result.params[:-1]
alpha = abs(result.params[3])

counterfactual_shares = comp__outside_shares(observ,prices,alpha,beta)

print(counterfactual_shares)


## Average decrease in uninsured rate
print(shares[['outside_fitted_share']].mean()-counterfactual_shares[['outside_fitted_share']].mean())

In [ ]:
#consumer surplus
#1/alpha *  ( sum(np.exp(delta1_j)) - sum(np.exp(delta0_j)) )
#I used the formula 1/alpha *  (ln( sum(np.exp(delta1_j))) - ln(sum(np.exp(delta0_j))) )
def comp_cs(x,p,alpha,beta,labels=False):
    #compute exp(delta_j)
    x = x.copy()
    x['exp_delta'] =  np.exp ( np.matmul(x[['Network Score','Satisfaction Score','PPO']],beta) - alpha*p )
    #print x['exp_delta']
    
    #compute log(exp(delta))
    sum_delta = x.groupby('Market_ID').sum()
    sum_delta['sum_exp_delta_log'] = np.log(sum_delta['exp_delta'])
    
    x = pd.merge(x, sum_delta[['sum_exp_delta_log']], 
                right_index=True, left_on = 'Market_ID')

    #compute cs
    x['consumer_surplus'] = (1/alpha)*x['sum_exp_delta_log']
    
    return x[['Market_ID','consumer_surplus']]
    
   

#set up parameters 

observ = data[['Plan_ID','Market_ID','Network Score','Satisfaction Score','PPO']]
prices = np.array( data['Premium'] )

beta = result.params[:-1]
alpha = abs(result.params[3])

cs = comp_cs(observ,prices,alpha,beta)

# counterfactual parameters 

prices = np.array(new_prices)
cscounterfactual = comp_cs(observ,prices,alpha,beta)


print(cscounterfactual.groupby('Market_ID').mean())
print(cs.groupby('Market_ID').mean())

#Average increase. First averaging within market, then over all markets
# because markets have different number of providers
print(cscounterfactual.groupby('Market_ID').mean().mean()[0]-cs.groupby('Market_ID').mean().mean()[0])
